<a href="https://colab.research.google.com/github/rkrissada/100DayOfMLCode/blob/master/day_042_sampled_softmax_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import MINST data
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Parameters
learning_rate = 0.001
training_epochs = 6
batch_size = 100
display_step = 1

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
import tensorflow as tf

# Network Parameters
n_hidden_1 = 256  # 1st layer number of features
n_input = 784     # MNIST data input (img shape: 28*28)
n_classes = 10    # MNIST total classes (0-9 digits)    

# Dependent & Independent Variable Placeholders
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Weights and Biases
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Super simple model builder
def tiny_perceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer   

# Create the model
pred = tiny_perceptron(x, weights, biases)

In [5]:
# Training cycle
mode = "train"

def get_loss_func(mode):
    
    # Set up loss function inputs and inspect their shapes
    
    w = tf.transpose(weights['out'])
    b = biases['out']
    inputs = pred
    num_sampled = 3
    num_true = 1
    num_classes = n_classes
    
    #
    if mode == "train":
        # Convert one-hot to index
        labels = tf.reshape(tf.argmax(y, 1), [-1,1])
        
        loss = tf.nn.sampled_softmax_loss(
                         weights=weights['out'],
                         biases=b,
                         labels=labels,
                         inputs=inputs,
                         num_sampled=num_sampled,
                         num_true=num_true,
                         num_classes=num_classes,
                         partition_strategy="div")

    # Evaluation Mode
    elif mode == "eval":
        labels_one_hot = y
        
        logits = tf.matmul(inputs, tf.transpose(weights['out']), name='Eval_Logits') # ERROR HERE!!!!
        logits = tf.nn.bias_add(logits, biases['out'])
        loss = tf.nn.softmax_cross_entropy_with_logits(
            labels=labels_one_hot,
            logits=logits)
    
    return loss

# Training Loss
loss = get_loss_func(mode)

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Initializing the variables
init = tf.global_variables_initializer()
# Launch the graph
sess = tf.InteractiveSession()
sess.run(init)

Instructions for updating:
Use tf.cast instead.


In [6]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c = sess.run([optimizer, loss], feed_dict={x: batch_x,
                                                      y: batch_y})
        
        # Compute average loss
        avg_cost += c / total_batch
    # Display logs per epoch step
    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(np.mean(avg_cost)))
print("Optimization Finished!")

# Test model
#correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
#correct_prediction = tf.equal(tf.nn.softmax(tf.matmul(pred, weights['out']) + biases['out']), tf.argmax(y,1))
# Calculate accuracy
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

#print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 38.124340057
Epoch: 0002 cost= 9.415617943
Epoch: 0003 cost= 5.918379307
Epoch: 0004 cost= 4.357186794
Epoch: 0005 cost= 3.291755676
Epoch: 0006 cost= 2.615612507
Optimization Finished!
